In [1]:
import sys
import json
from tqdm import tqdm
from typing import Dict
import numpy as np
import torch
import os
from time import time
import gc

# TO CHANGE
BASEDIR = "/home/dzigen/Desktop/PersonalAI/Personal-AI"
# TO CHNAGE

sys.path.insert(0, BASEDIR)

EVAL_DATADIR = '../../data/qa_eval'

from src.qa_pipeline import QAPipeline, QAPipelineConfig
from src.qa_pipeline.query_parser import QueryLLMParser
from src.qa_pipeline.knowledge_comparator import KnowledgeComparator
from src.qa_pipeline.knowledge_retriever import KnowledgeRetriever, KnowledgeRetrieverConfig, AStarGraphSearchConfig
from src.qa_pipeline.knowledge_retriever.AStarTripletsRetriever import AStarMetricsConfig
from src.qa_pipeline.answer_generator import QALLMGenerator

from src.agents.private import GigaChatAgent
from src.knowledge_graph_model import KnowledgeGraphModel
from src.neo4j_functions import Neo4jConnection
from src.embedding_functions import EmbeddingsDatabaseConnection, EmbeddingsDatabaseConnectionConfig, VectorDBConnectionConfig, EmbedderModelConfig

from src.utils import ReaderMetrics
from src.embedding_functions import ChromaConnection, VectorDBConnectionConfig, EmbeddingsDatabaseConnectionConfig
from src.agents.private import GigaChatAgent
from src.utils.data_structs import NodeType

/home/dzigen/Desktop/PersonalAI/Personal-AI/pai_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
agent = GigaChatAgent()
kg_model = KnowledgeGraphModel(
    graph_db=Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="password", db_name="diaasq3"),
    embeddings_db=EmbeddingsDatabaseConnection(
        config=EmbeddingsDatabaseConnectionConfig(
            nodes_db_config=VectorDBConnectionConfig(
                path="../../data/graph_structures/vectorized_nodes/v11/densedb",
                db_name="vectorized_nodes"
            ),
            triplets_db_config=VectorDBConnectionConfig(
                path="../../data/graph_structures/vectorized_triplets/v7/densedb",
                db_name="vectorized_triplets"
            ),
            embedder_config=EmbedderModelConfig(
                model_name_or_path="../../models/intfloat/multilingual-e5-small"
            )
        )
    )
)

No sentence-transformers model found with name ../../models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


In [3]:
QUERIES = [
    "Which device is better in battery life: iPhone11 Pro Max or Xiaomi 11?", # быстро X
    "The majority of speakers have positive, neutral or negative sentiment about connection of Apple?", # ~1 min X
    "The majority of speakers have positive, neutral or negative sentiment about signal of Apple?", # ~28sec X
    "Kayla has positive, negative or neutral opinion about video of 10PRO on 25.11.2020?", # ~ 2 min X
    "What Jessica's opinion (positive, negative or neutral) about signal of Apple was dominant during using Apple?", # ~ 25sec X
    "What opinion (positive, negative or neutral) about scheduling of IQOO9 was last during Zachary's experience of IQOO9?", # ~ 38sec
    "Do Jane and Jonathan have any common devices (which Jane and Jonathan both use)? If so, list common devices. Otherwise, answer 'No'.", # ~8sec X
    "Do Maria and Tyler prefer the same device manufacturer? If so, list common manufacturers. Otherwise, answer 'No'.", # ~42sec X
    "Whose opinions from Freda and Bruce about devices are most similar to Kayla's?", # ~26sec X
    "Whose opinions from Linda and Arianna about manufacturers are most similar to Hugh's?", # 1.20min X
    "Which people have positive opinion about display of Mi 10pro on 22.12.2018?", # ~ 2 min X
    "Which people have negative opinion about video of 10PRO on 25.11.2020?" # ~ 2 min X
    ]

### QA-пайплан целиком

In [4]:
qa_pipeline = QAPipeline(kg_model, agent)
#qa_pipeline.answer(QUERY)

In [5]:
agent.generate("2 + 2 = ")

'4'

In [ ]:
for query in tqdm(QUERIES):
    s_time = time()
    answer = qa_pipeline.answer(query)
    e_time = time()
    print(answer, e_time - s_time)

In [5]:
qa_files = os.listdir(EVAL_DATADIR)

In [26]:
for qa_file in qa_files[6:7]:
    print(qa_file)
    with open(f"{EVAL_DATADIR}/{qa_file}", 'r', encoding='utf-8') as fd:
        data = json.loads(fd.read())

    gen_answers = []
    process = tqdm(data)
    for qa_pair in process:
        gen_answer = qa_pipeline.answer(qa_pair['question'])
        gen_answers.append({"generated_answer": gen_answer})
        process.set_postfix({'target': qa_pair['answer'], 'generated': gen_answer})
    
    with open(f"./logs/generated/{qa_file}", 'w', encoding='utf-8') as fd:
        fd.write(json.dumps(gen_answers, indent=1, ensure_ascii=False))

last_opinion.json


100%|██████████| 3157/3157 [11:10:19<00:00, 12.74s/it, target=positive, generated=Not available (since there is no opinion about the hand feel of S8 from Megan)]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

### QA-пайплайн по частям

In [40]:
# stage 1
q_parser = QueryLLMParser(agent)
qparser_out = q_parser.extract_entities(QUERIES[11])
print(qparser_out.entities)

['people', 'opinion', 'video', '10PRO', '25.11.2020']


In [ ]:
# stage 2
k_comparator = KnowledgeComparator(kg_model)
k_comparator.link_kgnodes_to_query(qparser_out)

for item in qparser_out.linked_nodes:
    print(item)

In [ ]:
# stage 3
k_retriever = KnowledgeRetriever(kg_model, config=KnowledgeRetrieverConfig(
    graph_retriever_config=AStarGraphSearchConfig(metrics_config=AStarMetricsConfig(
        nodes_distances_path = '../../data/graph_structures/vectorized_nodes/v8/nodes_distances_matrix',
        nodes_short_paths_file = '../../data/graph_structures/graph_short_paths/stage2/v2/distances_matrix'
    ))))
kretriever_out = k_retriever.retrieve(qparser_out)

for item in kretriever_out:
    print(item)

In [ ]:
# stage 4
qa_generator = QALLMGenerator(agent)
contexts = qa_generator.formate_context(kretriever_out)
qagenerator_out = qa_generator.generate(qparser_out.query, contexts)

print(contexts)
print()
print(qagenerator_out)

### Оценка качества

In [14]:
METRICS = ReaderMetrics(base_dir="../..", bs_model_path="google/electra-base-discriminator")

Loading Meteor...
Loading ExactMatch
Loading BertScore


In [10]:
save_log_tmpdata_dir = "./logs/scores"
gen_answers_dir = "./logs/generated"

In [11]:
def save_json(data: Dict[str, object], save_path: str):
    dump = json.dumps(data, ensure_ascii=False, indent=1)
    with open(save_path, 'w', encoding='utf-8') as fd:
        fd.write(dump)

def load_json(load_path: str) -> Dict[str,object]:
    with open(load_path, 'r', encoding='utf-8') as fd:
        data = json.loads(fd.read())
    return data

def round5(number: float) -> float:
    return round(number, 5)


def measure_quality_from_answers_file(qa_file: str, gen_answers_dir: str, save_log_tmpdata_dir: str):
    print(qa_file)
    generated_answers_data = load_json(f"{gen_answers_dir}/{qa_file}")
    target_answers_data = load_json(f"{EVAL_DATADIR}/{qa_file}")
    
    gen_answers = list(map(lambda item: item['generated_answer'], generated_answers_data)) 
    trgt_answers = list(map(lambda item: item['answer'], target_answers_data))[:len(gen_answers)]

    b1_scores = METRICS.bleu1(gen_answers, trgt_answers)
    b2_scores  = METRICS.bleu2(gen_answers, trgt_answers)
    rl_scores = METRICS.rougel(gen_answers, trgt_answers)
    m_scores = METRICS.meteor(gen_answers, trgt_answers)
    em_scores = METRICS.exact_match(gen_answers, trgt_answers)
    bs_scores = METRICS.bertscore(gen_answers, trgt_answers)

    scores = {
        'BLEU1': str(round5(np.mean(b1_scores))),
        'BLEU2': str(round5(np.mean(b2_scores))),
        'METEOR': str(round5(np.mean(m_scores))),
        'RougeL': str(round5(np.mean(rl_scores))),
        'ExactMatch': str(round5(np.mean(em_scores))),
        'BertScore': {k: str(round5(float(v.mean()))) for k, v in bs_scores.items() if k != 'hash'}
    }

    torch.cuda.empty_cache()
    gc.collect()

    save_json(scores, f"{save_log_tmpdata_dir}/{qa_file}")

In [12]:
generated_files = os.listdir(gen_answers_dir)

In [27]:
for gen_file in ['last_opinion.json']:
    measure_quality_from_answers_file(gen_file, gen_answers_dir, save_log_tmpdata_dir)

last_opinion.json
